In [1]:
import csv
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
data= pd.read_csv(r'C:\New_partition\Project_Pattern_Recognition\amazon_books_review.csv')

In [5]:
df_cleaned = data.head(100000)
df_cleaned['star_rating'].count()
df_cleaned['star_rating'].unique()
df_cleaned['star_rating'].value_counts()
indexNames = df_cleaned[ df_cleaned['star_rating'] == 'star_rating' ].index
df_cleaned.drop(indexNames,inplace=True)
df_cleaned.dropna(inplace=True)

In [6]:
stop=stopwords.words('english')
df_cleaned['star_rating']=df_cleaned['star_rating'].astype(float).astype(int)
df_cleaned['review_body']=df_cleaned['review_body'].str.replace('<br />', r'')
df_cleaned['review_body']=df_cleaned['review_body'].str.replace("[,'!;.–`:?><-]",'')
#df_cleaned['review_body']=df_cleaned['review_body'].str.replace("[!#$%&'()*+, -./:;<=>?@[\]^_`{|}~]",'')
df_cleaned['review_body']=df_cleaned['review_body'].str.lower()
#df_cleaned=df_cleaned[df_cleaned.review_body.str.split(' ').str.len()<300]
df_cleaned['review_body']=df_cleaned.review_body.str.replace('[^a-zA-Z ]', '')
df_cleaned.dropna(inplace=True)
df_cleaned.isna().sum()
df_cleaned['review_body']=df_cleaned['review_body'].apply(lambda x: ' '.join([item for item in x.split() if item not in stop]))

In [7]:
def partition(x):
    if x<3:
        return 'Negative'
    elif x>3:
        return 'Positive'
    return 'Neutral'

actualScore = df_cleaned['star_rating']
positiveNegative = actualScore.map(partition)
df_cleaned['star_rating'] = positiveNegative

In [8]:
X_train=df_cleaned['review_body']
y_train=df_cleaned['star_rating']

In [9]:
data_test= pd.read_csv(r'C:\New_partition\Project_Pattern_Recognition\test.csv')
df_cleaned = data_test
indexNames = df_cleaned[ df_cleaned['ratings'] == 'ratings' ].index
df_cleaned.drop(indexNames,inplace=True)
df_cleaned.dropna(inplace=True)

In [10]:
df_cleaned['review_body']=df_cleaned['review_body'].str.replace('<br />', r'')
df_cleaned['review_body']=df_cleaned['review_body'].str.replace("[,'!;.–`:?><-]",'')
#df_cleaned['review_body']=df_cleaned['review_body'].str.replace("[!#$%&'()*+, -./:;<=>?@[\]^_`{|}~]",'')
df_cleaned['review_body']=df_cleaned['review_body'].str.lower()
#df_cleaned=df_cleaned[df_cleaned.review_body.str.split(' ').str.len()<300]
df_cleaned['review_body']=df_cleaned.review_body.str.replace('[^a-zA-Z ]', '')
df_cleaned.dropna(inplace=True)
df_cleaned.isna().sum()
df_cleaned['review_body']=df_cleaned['review_body'].apply(lambda x: ' '.join([item for item in x.split() if item not in stop]))
X_test=df_cleaned['review_body']
y_test=df_cleaned['ratings']

In [11]:
clf = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', LogisticRegression()),
])
clf = clf.fit(X_train, y_train)
accuracy_train=accuracy_score(y_train, clf.predict(X_train))
accuracy_test=accuracy_score(y_test, clf.predict(X_test))
print('Accuracy on train %f' %accuracy_train)
print('Accuracy on test %f' %accuracy_test)

Accuracy on train 0.953841
Accuracy on test 0.834538


In [30]:
clf = Pipeline([
    ('vect', CountVectorizer(analyzer='word', ngram_range=(1, 2))),
    ('clf', LogisticRegression(max_iter=500,C=0.1,random_state=40,solver='newton-cg')),
])
clf = clf.fit(X_train, y_train)
accuracy_train=accuracy_score(y_train, clf.predict(X_train))
accuracy_test=accuracy_score(y_test, clf.predict(X_test))
print('Accuracy on train %f' %accuracy_train)
print('Accuracy on test %f' %accuracy_test)

Accuracy on train 0.984243
Accuracy on test 0.856599


In [31]:
from sklearn.tree import DecisionTreeClassifier
clf = Pipeline([
    ('vect', CountVectorizer(analyzer='word', ngram_range=(1, 2))),
    ('clf', DecisionTreeClassifier()),
])
clf = clf.fit(X_train, y_train)
accuracy_train=accuracy_score(y_train, clf.predict(X_train))
accuracy_test=accuracy_score(y_test, clf.predict(X_test))
print('Accuracy on train %f' %accuracy_train)
print('Accuracy on test %f' %accuracy_test)

Accuracy on train 0.999910
Accuracy on test 0.770068


In [32]:
from sklearn.svm import LinearSVC
clf = Pipeline([
    ('vect',TfidfVectorizer()),
    ('clf',LinearSVC()),
])
clf = clf.fit(X_train, y_train)
accuracy_train=accuracy_score(y_train, clf.predict(X_train))
accuracy_test=accuracy_score(y_test, clf.predict(X_test))
print('Accuracy on train %f' %accuracy_train)
print('Accuracy on test %f' %accuracy_test)

Accuracy on train 0.959617
Accuracy on test 0.848298
